## General formatting

In [1]:
import requests
import pandas as pd
import os

# get EIA API key from environment variable
EIA_API_KEY = os.getenv("EIA_API_KEY")
EIA_BASE_URL = "https://api.eia.gov/v2/"
api_path = 'electricity/rto/region-data/'
get_request = EIA_BASE_URL + api_path +\
    'data/&data[]=value' + '?api_key=' + EIA_API_KEY 

response = requests.get(get_request)
data = response.json()
df = pd.DataFrame(data['response']['data'])
print(df.head())

          period respondent                                 respondent-name  \
0  2025-12-31T08        AVA                              Avista Corporation   
1  2025-12-31T08       BANC      Balancing Authority of Northern California   
2  2025-12-31T08       BPAT                 Bonneville Power Administration   
3  2025-12-31T08        CAL                                      California   
4  2025-12-31T08       CHPD  Public Utility District No. 1 of Chelan County   

  type                  type-name  
0   DF  Day-ahead demand forecast  
1   DF  Day-ahead demand forecast  
2   DF  Day-ahead demand forecast  
3   DF  Day-ahead demand forecast  
4   DF  Day-ahead demand forecast  


In [2]:
# Extract the API key from the environment variable
eia_api_key = os.getenv('EIA_API_KEY')

# Create the full URL path
api_url = "https://api.eia.gov/v2/"
api_path = "electricity/rto/region-data/"
api_url_path = api_url + api_path + "data/&data[]=value"
full_path = api_url_path + "?api_key=" + eia_api_key

# Make the request
data = requests.get(full_path).json()

df = pd.DataFrame(data["response"]["data"])
print(df.head())

          period respondent                                 respondent-name  \
0  2025-12-31T08        AVA                              Avista Corporation   
1  2025-12-31T08       BANC      Balancing Authority of Northern California   
2  2025-12-31T08       BPAT                 Bonneville Power Administration   
3  2025-12-31T08        CAL                                      California   
4  2025-12-31T08       CHPD  Public Utility District No. 1 of Chelan County   

  type                  type-name  
0   DF  Day-ahead demand forecast  
1   DF  Day-ahead demand forecast  
2   DF  Day-ahead demand forecast  
3   DF  Day-ahead demand forecast  
4   DF  Day-ahead demand forecast  


In [4]:
df.columns

Index(['period', 'respondent', 'respondent-name', 'type', 'type-name'], dtype='object')

In [ ]:
# example
df = pd.read_csv('data/data.csv')
ts = df[['period', 'value']]
ts['period'] = pd.to_datetime(ts['period'])
ts = ts.sort_values('period')

ts = ts.rename(columns={'period': 'ds', 'value': 'y'})
ts['unique_id'] = 1        

from statsforecast import StatsForecast

fig = StatsForecast.plot(ts, engine="plotly")
fig.show()

- data preparation
- model(s) training
- model(s)  evaluation

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import (
        DynamicOptimizedTheta, SeasonalNaive, AutoARIMA, HoltWinters, MSTL
    )

from utilsforecast.plotting import plot_series 

In [ ]:
test_length = 72
train_end = end-datetime.timedelta(hours=test_length)
train = ts[(ts['ds'] <= train_end)]
test = ts[(ts['ds'] > train_end)]

plot_series(train, engine="plotly")
plot_series(test, engine="plotly")

In [ ]:
# instantiate models with hourly seasonality
from statistics import mean

from cv2 import sqrt


auto_arima = AutoARIMA()
s_naive = SeasonalNaive(season_length=24)
theta = DynamicOptimizedTheta(season_length=24)

# instantiate models with daily and weekly seasonality
mstl1 = MSTL(season_length=[24, 24*7],
             trend_forecaster=AutoARIMA(),
             alias="MSTL-ARIMA_trend")

mstl2 = MSTL(season_length=[24, 24*7],
             trend_forecaster=HoltWinters(),
             alias="MSTL-HW_trend")

# store models in a list
stats_models = [auto_arima, s_naive, theta, mstl1, mstl2]

# instantiate StatsForecast object
sf = StatsForecast(models=stats_models, 
                   freq='H',
                   fallback_model=AutoARIMA(), 
                   n_jobs=-1)

# create forecasts
forecasts = sf.forecast(df=train, 
                        h=72, 
                        level=95 #[80, 95]
                        )

# plot forecasts
p = sf.plot(test, forecasts, engine="plotly", level=[95])
p.update_layout(weight=400)
p.show()

# model evaluation
def mape(y_true, y_pred):
    mape = mean(abs((y_true - y_pred) / y_true))
    return mape

def rmse(y_true, y_pred):
    rmse = sqrt(mean((y_true - y_pred) ** 2))
    return rmse

def coverage(y_true, lower_bound, upper_bound):
    coverage = mean((y_true >= lower_bound) & (y_true <= upper_bound))
    return coverage

# merge forecasts with test set
fc = forecasts.merge(test, on=['unique_id', 'ds'], how='left')
fc_performance = None

for i in [str(m) for m in stats_models]:
    m = mape(y = fc['y'], y_pred = fc[i])
    r = rmse(y = fc['y'], y_pred = fc[i])
    c = coverage(y = fc['y'], 
                 lower_bound = fc[f'{i}_lower_95'], 
                 upper_bound = fc[f'{i}_upper_95'])
    perf = {'model': i, 
            'mape': m, 
            'rmse': r, 
            'coverage': c}
    if fc_performance is None:
        fc_performance = pd.DataFrame(perf, index=[0])
    else:
        fc_performance = pd.concat([fc_performance, pd.DataFrame(perf, index=[0])])

fc_performance.reset_index(drop=True, inplace=True)
print(fc_performance.sort_values('rmse'))

In [ ]:
# using mlforecast package
from mlforecast import MLForecast
import datetime
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from mlforecast.utils import PredictionIntervals

# Define the ML models
ml_models = [LGBMRegressor(),  XGBRegressor(), LinearRegression()]

# Set up the MLForecast object with models and frequency
mlf = MLForecast(
    models= ml_models,  
    freq='h', 
    lags=list(range(1, 24)), 
    date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week', 'hour'])


ml_models = [LGBMRegressor(), XGBRegressor(), LinearRegression()]

mlf = MLForecast(
    models= ml_models,  
    freq='h', 
    lags=list(range(1, 24)), 
    date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week', 'hour'])

# Fit the model to the training data
mlf.fit(df=train,  fitted=True, 
prediction_intervals=PredictionIntervals(n_windows=5, h=72, method="conformal_distribution"))

# Generate forecasts for the next 72 hours with 95% confidence
ml_forecast = mlf.predict(72, level=[95])

# print(ml_forecast.head())

# Combine the data
fc = ml_forecast.merge(test, how="left", on="ds")
fc_performance = None

for model in ["LGBMRegressor", "XGBRegressor", "LinearRegression"]:
    m = mape(y=fc["y"], yhat=fc[model])
    
    # Calculate RMSE
    r = rmse(y=fc["y"], yhat=fc[model])
    c = coverage(y=fc["y"], lower=fc[model + "-lo-95"], upper=fc[model + "-hi-95"])

    perf = {"model": model, "mape": m, "rmse": r, "coverage": c}
    if fc_performance is None:
        fc_performance = pd.DataFrame([perf])
    else:
        fc_performance = pd.concat([fc_performance, pd.DataFrame([perf])])

# Sort the performance metrics by rmse
print(fc_performance.sort_values("rmse"))

# Plot the forecast results
fig = plot_series(test, ml_forecast, level=[95], engine="plotly").update_layout(height=400)
fig.show()